In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint
tf.set_random_seed(777)
from tensorflow.python.layers import core

In [2]:
datas= 'data_samples/'
# vocabulary = 'vocab.csv'
# vocabs = []
# with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
#     words = csv.reader(vocab)
#     for i, word in enumerate(words):
#         vocabs.append(word[0])
# print(len(vocabs))
# word_to_id = {word: i for i, word in enumerate(vocabs)}
# id_to_word = {i:word for i, word in enumerate(vocabs)}
pad_token, pad_id = "§PAD§", 0
oov_token, oov_id = "§OOV§", 1
indent_token = "§<indent>§"
dedent_token = "§<dedent>§"
number_token = "§NUM§"
start_token = "§GO§"
end_token = "§END§"

In [3]:
word_to_id='data_samples/mapping.map'
with open(word_to_id, 'rb') as f:
    word_to_id = pickle.load(f)
len(word_to_id)

14760

In [4]:
# word_to_id.update(end_token)
word_to_id[end_token]=len(word_to_id)-1
word_to_id[start_token] = len(word_to_id)
inv_map = {v:k for k, v in word_to_id.items() }

In [5]:
len(word_to_id), len(inv_map)

(14762, 14761)

In [6]:
hidden_size=100
sequence=5
embedding_dim=50
attention_size = 50
batch_size=8
vocab_size=len(word_to_id)
vocab_size

14762

In [7]:
def get_model_inputs():
    X = tf.placeholder(tf.int32, [None, sequence], name='inputs_xdata')
    # Y = tf.placeholder(tf.float32, [None, sequence], name='targets_ydata')
    Y = tf.placeholder(tf.int32, [None, sequence], name='targets_ydata')
#     seq_len = tf.placeholder(tf.int32, [None], name='seq_len')
    lr = tf.placeholder(tf.float32, name='learning_rate')

#     embedding_variable = tf.Variable(tf.random_uniform([vocab_size, embedding_dim],-1.0,1.0), trainable=True)
#     batch_embedded = tf.nn.embedding_lookup(embedding_variable, X)
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return X, Y, lr, target_sequence_length, source_sequence_length, max_target_sequence_length

In [8]:
def encoding_layer(input_data, rnn_size, num_layers,
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state

In [9]:
def process_decoder_input(target_data, word_to_int, batch_size):
    ending = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size,1], word_to_int[start_token]), ending], 1)
    return dec_input

In [10]:
def decoding_layer(target_letter_to_int, decoding_embedding_size, num_layers, rnn_size,
                   target_sequence_length, max_target_sequence_length, enc_state, dec_input):
    
    target_vocab_size = len(target_letter_to_int)
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))

    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])

    output_layer = core.Dense(target_vocab_size,
                              kernel_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.1)
                             )
    
    with tf.variable_scope('decode'):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length = target_sequence_length,
                                                            time_major=False
                                                           )
        
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           enc_state,
                                                           output_layer
                                                          )
        
        training_decoder_output,final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                                                        impute_finished=True,
                                                                                                        maximum_iterations=max_target_sequence_length
                                                                                                       )
        
    with tf.variable_scope('decode', reuse=True):
        start_tokens = tf.tile(tf.constant([target_letter_to_int[start_token]],dtype=tf.int32), [batch_size], name='start_tokens')
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                    start_tokens,
                                                                    target_letter_to_int[end_token]
                                                                   )
        
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                            inference_helper,
                                                            enc_state,
                                                            output_layer
                                                           )
        
        inference_decoder_output, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                                                          impute_finished=True,
                                                                                                          maximum_iterations=max_target_sequence_length
                                                                                                         )
        
        print('training_decoder_output.rnn_output, inference_decoder_output.sample_id', training_decoder_output.rnn_output, inference_decoder_output.sample_id)
        print('training_decoder_output, inference_decoder_output', training_decoder_output, inference_decoder_output)
    return training_decoder_output, inference_decoder_output

In [11]:
def seq2seq_model(input_data, targets, target_sequence_length,max_target_sequence_length,
                  source_sequence_length,source_vocab_size, target_vocab_size, enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers
                 ):

    _, enc_state = encoding_layer(input_data=input_data, rnn_size=rnn_size, num_layers=num_layers,
                                  source_sequence_length = source_sequence_length,
                                  source_vocab_size = source_vocab_size,
                                  encoding_embedding_size=embedding_dim
                                 )
    
    dec_input = process_decoder_input(targets, word_to_id, batch_size)

    training_decoder_output, inference_decoder_output = decoding_layer(target_letter_to_int=word_to_id,
                                                                       decoding_embedding_size=embedding_dim,
                                                                       num_layers = num_layers,
                                                                       rnn_size=rnn_size,
                                                                       target_sequence_length=target_sequence_length,
                                                                       max_target_sequence_length=max_target_sequence_length,
                                                                       enc_state=enc_state,
                                                                       dec_input=dec_input
                                                                      )
    
    return training_decoder_output,inference_decoder_output

In [12]:
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():

    # Load the model inputs    
    input_data, targets, lr, target_sequence_length, source_sequence_length, max_target_sequence_length = get_model_inputs()
    print('input_data fetched ')
    # Create the training and inference logits
    training_decoder_output, inference_decoder_output = seq2seq_model(input_data=input_data, 
                                                                      targets=targets, 
                                                                      target_sequence_length = target_sequence_length, 
                                                                      max_target_sequence_length = max_target_sequence_length, 
                                                                      source_sequence_length = source_sequence_length,
                                                                      source_vocab_size = len(word_to_id),
                                                                      target_vocab_size = len(word_to_id),
                                                                      enc_embedding_size = embedding_dim, 
                                                                      dec_embedding_size = embedding_dim, 
                                                                      rnn_size = hidden_size, 
                                                                      num_layers = 1)
    
    print('seq2seq model created! ')
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_decoder_output.rnn_output, 'logits')
    predictions_sample_id = tf.identity(inference_decoder_output.sample_id, name='predictions_sample_id')
    predictions_output = tf.identity(inference_decoder_output.rnn_output, name='predictions_output')

    # Create the weights for sequence_loss
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):

        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(training_logits, targets, masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
        print('optmization created!')

input_data fetched 
training_decoder_output.rnn_output, inference_decoder_output.sample_id Tensor("decode/decoder/transpose:0", shape=(8, ?, 14762), dtype=float32) Tensor("decode_1/decoder/transpose_1:0", shape=(8, ?), dtype=int32)
training_decoder_output, inference_decoder_output BasicDecoderOutput(rnn_output=<tf.Tensor 'decode/decoder/transpose:0' shape=(8, ?, 14762) dtype=float32>, sample_id=<tf.Tensor 'decode/decoder/transpose_1:0' shape=(8, ?) dtype=int32>) BasicDecoderOutput(rnn_output=<tf.Tensor 'decode_1/decoder/transpose:0' shape=(8, ?, 14762) dtype=float32>, sample_id=<tf.Tensor 'decode_1/decoder/transpose_1:0' shape=(8, ?) dtype=int32>)
seq2seq model created! 
optmization created!


In [13]:
# Ses = tf.Session(graph=train_graph)
with tf.Session(graph=train_graph) as Ses:
    print('session !')
    file_name_queue = tf.train.string_input_producer(['inputs'])
    reader = tf.TFRecordReader()
    _, single_x = reader.read(file_name_queue)
    print('gogo~')
    context_features = {
        "length": tf.FixedLenFeature([], dtype=tf.int64)
    }

    sequence_features = {
        "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
        "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
    }

    context_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=single_x,
                                                                       context_features=context_features,
                                                                       sequence_features=sequence_features
                                                                      )
    context_parsed['length'] = tf.cast(context_parsed['length'], tf.int32)
    batch_seq, batch_inputs = tf.contrib.training.bucket_by_sequence_length(context_parsed['length'],
                                                                            [sequence_parsed['tokens'], sequence_parsed['labels']],
                                                                            batch_size=batch_size,
                                                                            bucket_boundaries=[5,10],
                                                                            dynamic_pad=True,
                                                                            capacity=1000,
                                                                            num_threads=1
                                                                           )

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=Ses, coord=coord)
    Ses.run(tf.global_variables_initializer())
    epochs = 3
    for epoch in range(1, epochs):
        print('epoch', epoch)
        i = 0
        while True:

            seq_input_, batch_input_ = Ses.run([batch_seq, batch_inputs])

            feed_data = {input_data: batch_input_[0], targets: batch_input_[1], 
                         target_sequence_length:seq_input_, source_sequence_length:seq_input_, lr:0.0001}
            _, loss = Ses.run([train_op, cost], feed_data)
    #             print('started')
            
            i+=1
            print('{} loss {} '.format(i, loss))
            if i % 500 ==0:
                checkpoint = './seq2seq_test/seq2seq_test.ckpt'
                saver = tf.train.Saver()
                saver.save(Ses, checkpoint)
                print('model saved')
#                 break

session !
gogo~
epoch 1
1 loss 9.593586921691895 
2 loss 9.61474895477295 
3 loss 9.61766242980957 
4 loss 9.616276741027832 
5 loss 9.626627922058105 
6 loss 9.606064796447754 
7 loss 9.582280158996582 
8 loss 9.602997779846191 
9 loss 9.592549324035645 
10 loss 9.604501724243164 
11 loss 9.584922790527344 
12 loss 9.571645736694336 
13 loss 9.585071563720703 
14 loss 9.590543746948242 
15 loss 9.582094192504883 
16 loss 9.551584243774414 
17 loss 9.571601867675781 
18 loss 9.570917129516602 
19 loss 9.54769229888916 
20 loss 9.563138008117676 
21 loss 9.533966064453125 
22 loss 9.54394245147705 
23 loss 9.554178237915039 
24 loss 9.535240173339844 
25 loss 9.534360885620117 
26 loss 9.524557113647461 
27 loss 9.514538764953613 
28 loss 9.527448654174805 
29 loss 9.511944770812988 
30 loss 9.530111312866211 
31 loss 9.518041610717773 
32 loss 9.524301528930664 
33 loss 9.493040084838867 
34 loss 9.500757217407227 
35 loss 9.482926368713379 
36 loss 9.518899917602539 
37 loss 9.4989948

299 loss 5.829044342041016 
300 loss 5.534422874450684 
301 loss 4.938756942749023 
302 loss 5.0283966064453125 
303 loss 4.907893657684326 
304 loss 6.391371726989746 
305 loss 5.374349594116211 
306 loss 5.734320640563965 
307 loss 5.291292190551758 
308 loss 5.283541679382324 
309 loss 5.4264373779296875 
310 loss 5.374288082122803 
311 loss 4.779753684997559 
312 loss 5.423966884613037 
313 loss 4.987438201904297 
314 loss 5.453156471252441 
315 loss 4.966484069824219 
316 loss 5.200850486755371 
317 loss 5.188836097717285 
318 loss 4.78633451461792 
319 loss 5.291573524475098 
320 loss 5.350857257843018 
321 loss 5.1988654136657715 
322 loss 5.137431621551514 
323 loss 5.286803722381592 
324 loss 4.949399471282959 
325 loss 4.842171669006348 
326 loss 5.093766689300537 
327 loss 5.2104902267456055 
328 loss 4.6244001388549805 
329 loss 5.3695478439331055 
330 loss 4.68060302734375 
331 loss 4.851031303405762 
332 loss 5.154311180114746 
333 loss 4.997258186340332 
334 loss 4.24173

592 loss 4.181757926940918 
593 loss 5.576070308685303 
594 loss 5.5763421058654785 
595 loss 5.675232887268066 
596 loss 6.088115692138672 
597 loss 5.575209617614746 
598 loss 4.468138694763184 
599 loss 4.856194019317627 
600 loss 4.30712890625 
601 loss 4.584307670593262 
602 loss 4.1893110275268555 
603 loss 3.948763370513916 
604 loss 4.787625789642334 
605 loss 5.016982078552246 
606 loss 4.069695472717285 
607 loss 5.244447231292725 
608 loss 5.129562854766846 
609 loss 5.224910736083984 
610 loss 5.434578895568848 
611 loss 5.430225372314453 
612 loss 5.280104160308838 
613 loss 4.28798770904541 
614 loss 4.78061580657959 
615 loss 4.454530239105225 
616 loss 5.200340270996094 
617 loss 5.8956522941589355 
618 loss 5.824734687805176 
619 loss 5.507427215576172 
620 loss 4.853144645690918 
621 loss 4.621289253234863 
622 loss 4.678588390350342 
623 loss 4.532700538635254 
624 loss 4.980037689208984 
625 loss 5.558412075042725 
626 loss 5.04830265045166 
627 loss 4.6772866249084

886 loss 4.886833667755127 
887 loss 5.444912433624268 
888 loss 5.232644081115723 
889 loss 4.704957008361816 
890 loss 5.1692728996276855 
891 loss 5.148516654968262 
892 loss 5.645969390869141 
893 loss 4.458230495452881 
894 loss 4.78281307220459 
895 loss 4.564932823181152 
896 loss 5.277708530426025 
897 loss 4.9928879737854 
898 loss 5.140599727630615 
899 loss 5.499996185302734 
900 loss 4.926041603088379 
901 loss 5.121270179748535 
902 loss 5.175508975982666 
903 loss 5.074429512023926 
904 loss 4.88317346572876 
905 loss 5.157133102416992 
906 loss 5.113154888153076 
907 loss 4.750113487243652 
908 loss 4.928043365478516 
909 loss 4.914912223815918 
910 loss 4.330667018890381 
911 loss 5.093702793121338 
912 loss 4.475543975830078 
913 loss 4.8519086837768555 
914 loss 5.3751678466796875 
915 loss 5.97274923324585 
916 loss 5.455610752105713 
917 loss 5.389457702636719 
918 loss 4.82857608795166 
919 loss 5.99190092086792 
920 loss 5.351550102233887 
921 loss 5.4117846488952

ValueError: Cannot feed value of shape (8, 0) for Tensor 'inputs_xdata:0', which has shape '(?, 5)'